In [1]:
import ee
import sys
sys.path

['/usr/src/app/notebooks',
 '/usr/src/app',
 '/usr/src/app/gee_lib',
 '/usr/src/app/ex_ante_lib',
 '/usr/src/app/forestry_carbon_arr',
 '/usr/src/app/fastapi-gee-service',
 '/usr/local/lib/python310.zip',
 '/usr/local/lib/python3.10',
 '/usr/local/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/site-packages']

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from google.auth import default
creds, project = default()

In [4]:
# !gcloud projects list
# !gcloud config list
!gcloud auth list


     Credentialed Accounts
ACTIVE  ACCOUNT
*       muhammad.iqbal@treeo.one

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [5]:
!earthengine authenticate

Authenticate: Credentials already exist. Use --force to refresh.


In [6]:
!gcloud config set project ee-iwansetiawan

Updated property [core/project].


In [7]:
# !pip install earthengine-api --upgrade


In [8]:
import os
import ee
# from gee_lib.osi.auth import initialize_gee, check_gee_initialization

# ee.Initialize(project=os.getenv('GOOGLE_CLOUD_PROJECT'))
ee.Initialize(project='ee-iwansetiawan')

# Initialize GEE
# authenticate = initialize_gee(project_id=os.getenv('GOOGLE_CLOUD_PROJECT'), use_service_account=False)
# authenticate = initialize_gee(project_id='ee-iwansetiawan',use_service_account=True)

# Verify it's working
# if check_gee_initialization():
    # print("Ready to use GEE!")

In [9]:
!gcloud --version

# os.getenv('GOOGLE_CLOUD_PROJECT')

from google.cloud import storage
client = storage.Client()
print(client.project)


Google Cloud SDK 544.0.0
alpha 2025.10.17
beta 2025.10.17
bq 2.1.24
bundled-python3-unix 3.12.9
core 2025.10.17
gcloud-crc32c 1.0.0
gsutil 5.35
validation-469814


In [11]:
# !gcloud.storage.buckets.list

In [12]:
import json
import os

### need improvement on how to manage this config, we also in the config the path to the static file
# maybe we need to not put the path in the config.json, don't know yet
# Move to the parent directory of the current script
parent_dir = os.getcwd()

# Construct the absolute path to the JSON file in the 'input' folder
json_path= os.path.join(parent_dir, '00_input', 'balaban_conf.json')

# Read and load the JSON data from the file
with open(json_path, 'r') as file:
    config = json.load(file)

print('config---> ',config)

config--->  {'I_satellite': 'Sentinel', 'pca_scaling': 1, 'tileScale': 2, 'AOI_path': './00_input/LPHD Belaban Rayak.shp', 'OID_field_name': 'id', 'input_training': './00_input/traning_point_merged_belaban.shp', 'algo_ml_selected': 'gbm', 'date_start_end': ['2024-1-1', '2024-5-31'], 'project_name': 'belaban_rayak', 'super_pixel_size': 3, 'region': 'asia', 'pixel_number': 3, 'year_start_loss': 14, 'tree_cover_forest': 30, 'band_name_image': 'Class', 'cloud_cover_threshold': 60, 'crs_input': 'EPSG:4326', 'IsThermal': False, 'fcd_selected': 21, 'high_forest': 65, 'yrf_forest': 45, 'shrub_grass': 45, 'open_land': 30, 'ndwi_hi_sentinel': 0.05, 'ndwi_hi_landsat': 0.1, 'ndwi_hi_planet': -0.2}


In [13]:
## import modules
import osi
import pandas as pd
import geemap

from osi.utils.main import validate_aoi
# convert the modules for image collection (cloudless masking, compositing, reducer etc)
from osi.image_collection.main import ImageCollection
from osi.spectral_indices.spectral_analysis import SpectralAnalysis
from osi.spectral_indices.utils import normalization_100
from osi.hansen.historical_loss import HansenHistorical
from osi.classifying.assign_zone import AssignClassZone
from osi.legends.utils import convert_to_legend_items
from osi.legends.main import LegendsBuilder
from osi.obia.main import OBIASegmentation
from osi.ml.main import LandcoverML
from osi.fcd.main_fcd import FCDCalc
from osi.pca.pca_gee import PCA
from osi.hansen.historical_loss import HansenHistorical
from osi.classifying.assign_zone import AssignClassZone
from osi.arcpy.utils import safe_get_data_source


AOIt_shp_plot = geemap.shp_to_ee(config['AOI_path'])
crs_input = config['crs_input']
I_satellite = config['I_satellite']
project_name = config['project_name']

start_date = config['date_start_end'][0]
end_date = config['date_start_end'][1]

layer_name_image_mosaick = f'image_mosaick_result_ee_{project_name}'

AOI = AOIt_shp_plot
config['AOI'] = AOI

ndwi_hi = 0.1
if config['I_satellite'] == 'Landsat':
    ndwi_hi = config['ndwi_hi_landsat']
elif I_satellite == 'Sentinel':
    ndwi_hi = config['ndwi_hi_sentinel']
elif I_satellite == 'Planet':
    ndwi_hi = config['ndwi_hi_planet']

### Masking and overlay and area helper Make an image out of the AOI area attribute -> convert featurecollection into raster (image) for overlaying tools
OID = config['OID_field_name']
AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
    properties= [OID],
    reducer= ee.Reducer.first()
)

In [14]:
# now starting to do analysis
# initiate instance class for the image collection and later mosaicking
classInputCollection = ImageCollection(I_satellite=I_satellite,
                                       AOI=AOI, 
                                       date_start_end=config['date_start_end'], 
                                       cloud_cover_threshold = config['cloud_cover_threshold'],
                                       region=config['region'])

In [15]:
# run the method from image collection loaded, cloudless compositing until to image_mosaick
image_mosaick = classInputCollection.image_mosaick()

selecting Sentinel images


In [16]:
from gee_lib.osi.utils import generate_map_id

# Generate Map IDs
### Defining params visualization and layer name
layer_name = f'{I_satellite} mosaicked - {start_date}-{end_date} VegColor'

if config['I_satellite'] == 'Planet':
    # true color {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}
    # nir veg color {"bands":["red","nir","blue"],"min":0,"max":0.6,"gamma":1.5 }
    vis_param = {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}
else:
    vis_param = {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5}

layers_data = {'project_name': 'Sentinel-2 Cloudless Composite Analysis'}


print("✓ Map IDs generated for all layers")
# Display tile URLs
for layer_name, map_id_dict in generate_map_id({layer_name: vis_param}, {layer_name: image_mosaick})['all_mapid'].items():
    tile_url = map_id_dict['tile_fetcher'].url_format
    print(f"\n{layer_name.upper()}:")
    print(f"  Tile URL: {tile_url}")



✓ Map IDs generated for all layers
Generating GEE Map IDs...

SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR:
  Tile URL: https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/0b7b63a9cc7617c8c04b3e95499a7665-2df965e0f601ace8a7420e6c1f509fb4/tiles/{z}/{x}/{y}


In [17]:
## META data layers
layers_data = {
    'project_name': project_name,
}

# # Option 1: Simple tile URLs only
# map_layers = {
#     layer_name: map_ids[layer_name]['tile_fetcher'].url_format}

# Option 2 with more detail combination for generating wmts
map_layers = generate_map_id({layer_name: vis_param}, {layer_name: image_mosaick})['map_layers']
map_layers



Generating GEE Map IDs...


{'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/9fa9b34774552871bef7c376bcd33bb3-588f59f4712b390bd494488548271163/tiles/{z}/{x}/{y}',
  'name': 'Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor',
  'description': 'SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis',
  'vis_params': {'bands': ['swir2', 'nir', 'red'],
   'min': 0,
   'max': 0.6,
   'gamma': 1.5}}}

In [18]:
# import sys
# sys.path.append('/usr/src/app/fastapi-gee-service')
from gee_integration import process_gee_to_mapstore

# Import the AOI fix function properly
from gee_lib.osi.utils import process_aoi_geometry

def list_layers_to_wmts(map_layers, AOI):
    """
    Convert map layers to WMTS format
    
    Args:
        map_layers: Dictionary of map layers

    """
    
    # Fix 1: Use correct map layers
    print(f"\n🔍 Available map layers:")
    print(f"   map_layers: {len(map_layers)}")

    # Fix 2: Fix AOI coordinates using the existing function
    print(f"\n🔧 Fixing AOI coordinates...")
    AOI_geom = AOI.geometry()
    aoi_info = process_aoi_geometry(AOI_geom)

    print(f"✅ Fixed AOI Info:")
    print(f"   Bbox: {aoi_info['bbox']}")
    print(f"   Center: {aoi_info['center']}")

    print("🚀 Processing GEE Analysis to MapStore...")
    print(f"   Project: {layers_data['project_name']}")
    print(f"   Layers: {len(map_layers)}")
    print(f"   AOI: {aoi_info['bbox']}")

    # Process the complete integration
    result = process_gee_to_mapstore(
        map_layers=map_layers,
        project_name=layers_data['project_name'],
        aoi_info=aoi_info,
        clear_cache_first=True,
        fastapi_url="http://fastapi:8000"  # Internal Docker network , this should be in localhost, but we can override as well
    )
    return result

result = list_layers_to_wmts(map_layers, AOI)
result



🔍 Available map layers:
   map_layers: 1

🔧 Fixing AOI coordinates...
✅ Centroid calculated successfully with error margin 1


INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: belaban_rayak
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...
INFO:cache_manager:No existing catalog entries to check for duplicates
INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Registering with FastAPI: belaban_rayak_20251025_042950
INFO:gee_integration:✅ FastAPI registration successful: MapStore catalog updated successfully
INFO:gee_integration:Creating FastAPI proxy URLs for project: belaban_rayak_202

Calculated bbox from coordinates: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ AOI processed successfully:
   - Center: [110.46306597258281, -1.803818629688135]
   - Area: Unknown
   - BBox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ Fixed AOI Info:
   Bbox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
   Center: [110.46306597258281, -1.803818629688135]
🚀 Processing GEE Analysis to MapStore...
   Project: belaban_rayak
   Layers: 1
   AOI: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}


INFO:gee_utils:Found 0 GEE WMTS services
INFO:gee_utils:No GEE WMTS services to remove
INFO:gee_utils:Refreshing WMTS capabilities...
INFO:gee_utils:WMTS capabilities refreshed successfully
INFO:gee_utils:Updating MapStore WMTS service...
INFO:gee_utils:Successfully updated MapStore WMTS service: gee_analysis_wmts
INFO:gee_utils:Getting current WMTS layers...
INFO:gee_utils:Comprehensive WMTS refresh completed successfully - Found 1 layers
INFO:gee_integration:✅ MapStore WMTS configuration updated
INFO:gee_integration:   New layers found: 1


{'status': 'success',
 'project_id': 'belaban_rayak_20251025_042950',
 'project_name': 'belaban_rayak',
 'fastapi_registration': {'status': 'success',
  'message': 'MapStore catalog updated successfully',
  'layers_count': 1},
 'proxy_urls_creation': {'status': 'success',
  'message': 'Created 1 proxy URLs',
  'proxy_urls': {'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'proxy_url': 'http://fastapi:8000/tiles/belaban_rayak_20251025_042950/Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/9fa9b34774552871bef7c376bcd33bb3-588f59f4712b390bd494488548271163/tiles/{z}/{x}/{y}',
    'layer_name': 'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor',
    'description': 'SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis'}},
  'layers_count': 1},
 'wmts_configuration': {'status': 'success',
  'message': 'WMTS configuration updated successfully',
  'service_id'

In [19]:
## EXAMPLE TO PUSH GEE TO GEOJSON FAST API
import requests
import json

# Example GeoJSON data
geojson_data = AOI.getInfo() # important this ee.FeatureCollection should convert to geojson first
name_aoi = 'gee_aoi_dynamic'

link_fastapi = 'http://fastapi:8000/fc/'+name_aoi #only from container, not from localhost

# Push to API
response = requests.post(
    link_fastapi, # running in the notebook for now
    json=geojson_data
)

print('link_localhost: ', 'http://localhost:8001/fc/'+name_aoi)



link_localhost:  http://localhost:8001/fc/gee_aoi_dynamic


In [20]:
# !cat /usr/src/app/mapstore/config/localConfig.json

In [21]:
# Test the new TMS management system
from gee_integration import add_tms_layer_to_mapstore, list_gee_tms_layers, clear_all_gee_tms_layers
print("Adding TMS layer with Direct GEE TILES...")
# Get the first layer as an example
first_layer_name = list(map_layers.keys())[0]
first_layer_info = map_layers[first_layer_name]

# Add a TMS layer
tms_result = add_tms_layer_to_mapstore(
    layer_name=first_layer_name,
    layer_url=first_layer_info.get('tile_url', ''),
    use_fastapi_proxy=False, # change this to True if you want to use the fastapi proxy
    fastapi_pub_url="http://localhost:8001"
)

print(f"\n📊 TMS Proxy Result:")
print(f"   Status: {tms_result['status']}")
if tms_result['status'] == 'success':
    print(f"   Service ID: {tms_result['service_id']}")
    print(f"   Layer Title: {tms_result['layer_title']}")
    print(f"   TMS URL: {tms_result['tms_url']}")
    print(f"   Uses FastAPI Proxy: {tms_result['use_fastapi_proxy']}")


INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Adding TMS layer: Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor
INFO:gee_integration:✅ TMS layer added successfully
INFO:gee_integration:   Service ID: gee_tms_Sentinel_mosaicked_2024_1_1_2024_5_31_VegColor
INFO:gee_integration:   Layer: Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor
INFO:gee_integration:   URL: https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/9fa9b34774552871bef7c376bcd33bb3-588f59f4712b390bd494488548271163/tiles/{z}/{x}/{y}


Adding TMS layer with Direct GEE TILES...

📊 TMS Proxy Result:
   Status: success
   Service ID: gee_tms_Sentinel_mosaicked_2024_1_1_2024_5_31_VegColor
   Layer Title: Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor
   TMS URL: https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/9fa9b34774552871bef7c376bcd33bb3-588f59f4712b390bd494488548271163/tiles/{z}/{x}/{y}
   Uses FastAPI Proxy: False


In [22]:
list_gee_tms_layers()
# clear_all_gee_tms_layers()

INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Listing GEE TMS layers...
INFO:gee_integration:✅ Found 1 GEE TMS layers


{'status': 'success',
 'message': 'Found 1 GEE TMS layers',
 'tms_services': [{'service_id': 'gee_tms_Sentinel_mosaicked_2024_1_1_2024_5_31_VegColor',
   'layer_name': 'Sentinel_mosaicked_2024_1_1_2024_5_31_VegColor',
   'title': 'GEE TMS - Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor (Direct)',
   'url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/9fa9b34774552871bef7c376bcd33bb3-588f59f4712b390bd494488548271163/tiles/{z}/{x}/{y}',
   'type': 'tms',
   'format': 'image/png',
   'srs': 'EPSG:3857'}],
 'count': 1}

In [23]:
path_cog =  os.getenv('GCS_PATH')

In [24]:
## cog gcs to ee image
import geemap
drone_cog = geemap.load_GeoTIFF(path_cog)

In [25]:
type(drone_cog)

ee.image.Image

In [26]:
# !cat /root/.config/gcloud/application_default_credentials.json

In [36]:
map_layers = generate_map_id({layer_name: vis_param, 'drone_cog': {}}, {layer_name: image_mosaick, 'drone_cog': drone_cog})['map_layers']
map_layers

Generating GEE Map IDs...


{'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/3ac6b7ca35a28dcefe36a4bd2da6fb59-33000af4157fa68745e8f16960b5e028/tiles/{z}/{x}/{y}',
  'name': 'Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor',
  'description': 'SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis',
  'vis_params': {'bands': ['swir2', 'nir', 'red'],
   'min': 0,
   'max': 0.6,
   'gamma': 1.5}},
 'drone_cog': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/9af1e5e521a1824b9e4c5e1977b8b974-f51454276ed1f9f8acb22827ebf61d38/tiles/{z}/{x}/{y}',
  'name': 'Drone Cog',
  'description': 'DRONE_COG visualization from GEE analysis',
  'vis_params': {}}}

In [37]:
# re add
result = list_layers_to_wmts(map_layers, AOI)
result


🔍 Available map layers:
   map_layers: 2

🔧 Fixing AOI coordinates...
✅ Centroid calculated successfully with error margin 1


INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: belaban_rayak
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...
INFO:cache_manager:No existing catalog entries to check for duplicates
INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'drone_cog': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Registering with FastAPI: belaban_rayak_20251025_043651
INFO:gee_integration:✅ FastAPI registration successful: MapS

Calculated bbox from coordinates: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ AOI processed successfully:
   - Center: [110.46306597258281, -1.803818629688135]
   - Area: Unknown
   - BBox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ Fixed AOI Info:
   Bbox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
   Center: [110.46306597258281, -1.803818629688135]
🚀 Processing GEE Analysis to MapStore...
   Project: belaban_rayak
   Layers: 2
   AOI: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}


INFO:gee_utils:No GEE WMTS services to remove
INFO:gee_utils:Refreshing WMTS capabilities...
INFO:gee_utils:WMTS capabilities refreshed successfully
INFO:gee_utils:Updating MapStore WMTS service...
INFO:gee_utils:Successfully updated MapStore WMTS service: gee_analysis_wmts
INFO:gee_utils:Getting current WMTS layers...
INFO:gee_utils:Comprehensive WMTS refresh completed successfully - Found 2 layers
INFO:gee_integration:✅ MapStore WMTS configuration updated
INFO:gee_integration:   New layers found: 2


{'status': 'success',
 'project_id': 'belaban_rayak_20251025_043651',
 'project_name': 'belaban_rayak',
 'fastapi_registration': {'status': 'success',
  'message': 'MapStore catalog updated successfully',
  'layers_count': 2},
 'proxy_urls_creation': {'status': 'success',
  'message': 'Created 2 proxy URLs',
  'proxy_urls': {'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'proxy_url': 'http://fastapi:8000/tiles/belaban_rayak_20251025_043651/Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/3ac6b7ca35a28dcefe36a4bd2da6fb59-33000af4157fa68745e8f16960b5e028/tiles/{z}/{x}/{y}',
    'layer_name': 'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor',
    'description': 'SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis'},
   'drone_cog': {'proxy_url': 'http://fastapi:8000/tiles/belaban_rayak_20251025_043651/drone_cog/{z}/{x}/{y}',
    'original_url': 'https:

In [39]:
drone_name = list(map_layers.keys())[1]
drone_layer = map_layers[drone_name]
drone_layer

{'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/9af1e5e521a1824b9e4c5e1977b8b974-f51454276ed1f9f8acb22827ebf61d38/tiles/{z}/{x}/{y}',
 'name': 'Drone Cog',
 'description': 'DRONE_COG visualization from GEE analysis',
 'vis_params': {}}

In [40]:
# Add a TMS layer
tms_result = add_tms_layer_to_mapstore(
    layer_name=drone_name,
    layer_url=first_layer_info.get('tile_url', ''),
    use_fastapi_proxy=False, # change this to True if you want to use the fastapi proxy
    fastapi_pub_url="http://localhost:8001"
)

INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Adding TMS layer: drone_cog
INFO:gee_integration:✅ TMS layer added successfully
INFO:gee_integration:   Service ID: gee_tms_drone_cog
INFO:gee_integration:   Layer: Drone Cog
INFO:gee_integration:   URL: https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/9fa9b34774552871bef7c376bcd33bb3-588f59f4712b390bd494488548271163/tiles/{z}/{x}/{y}
